In [ ]:
%%html
<style>
.output_wrapper, .output { height:auto !important; max-height:10000px; } /* your desired max-height here */
.output_scroll {     box-shadow:none !important; webkit-box-shadow:none !important; }
.noborder, .noborder tr, .noborder th, .noborder td { border-collapse: collapse; border: 0px; border: none!important; }
</style>

In [ ]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [ ]:
platforms = ['demo8', 'poc1', 'poc2', 'nfv5']
#platforms = ['demo8', 'poc1' ]
platforms = ['demo8', 'poc2', 'nfv5']
platforms = ['nfv5']
platforms = ['poc2']
platforms = ['demo8']
platforms = ['poc1']

HTML_OPS, STATUSES, DISK_USAGE = getplatformStatuses(platforms)

In [ ]:
showPlatformStatuses(platforms, HTML_OPS, STATUSES, DISK_USAGE)

### TODO:
- use of openstack clientmanager for equivalent to "openstack service list" status

In [ ]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils